In [1]:
pip install statsmodels

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [3]:
trip_df = pd.read_csv('trip.csv')
weather_df = pd.read_csv('cleaned_weather_data.csv')

In [4]:
# Ensure datetime format for both datasets
trip_df['start_time'] = pd.to_datetime(trip_df['start_time'])
weather_df['time'] = pd.to_datetime(weather_df['time'])

# Align trip start times to the nearest previous half-hour
trip_df['half_hour'] = trip_df['start_time'].dt.floor('30min')

# Aggregate the trip counts for each half-hour interval
trip_counts = trip_df.groupby('half_hour').size().reset_index(name='trip_count')

# Merge with the weather data based on the half-hour intervals
# Left join so every weather record is kept, even if there were no trips in that interval
merged_df = pd.merge(weather_df, trip_counts, left_on='time', right_on='half_hour', how='left')

# Fill missing trip counts with 0 (no trips occurred in that interval)
merged_df['trip_count'] = merged_df['trip_count'].fillna(0)

# Optionally, drop the redundant 'half_hour' column
merged_df.drop(columns=['half_hour'], inplace=True)


In [5]:
merged_df.head()

,time,index,temperature_C,humidity,windspeed_kph,wind_direction_degres,wind_direction,pressure_mBar,visibility_km,wind_chill_C,...,temperature_bin,wind_chill_bin,humidity_bin,hour,time_of_day,month,month_str,bad_weather,severe_weather,trip_count
0,2015-01-01 00:00:00,0,-12.0,85,5.6,0,Variable,1037,5.0,-15.5,...,Very Cold,Very Cold,High,0,LateNight,1,Jan,0,0,12.0
1,2015-01-01 00:30:00,1,-11.0,85,3.7,0,Variable,1036,4.0,NaN,...,Very Cold,Hot,High,0,LateNight,1,Jan,0,0,8.0
2,2015-01-01 01:00:00,2,-10.0,86,3.7,0,Variable,1036,4.0,NaN,...,Very Cold,Hot,High,1,LateNight,1,Jan,0,0,12.0
3,2015-01-01 01:30:00,3,-10.0,86,0.0,0,North,1036,5.0,NaN,...,Very Cold,Hot,High,1,LateNight,1,Jan,0,0,11.0
4,2015-01-01 02:00:00,4,-9.0,79,0.0,0,North,1036,4.0,NaN,...,Very Cold,Hot,High,2,LateNight,1,Jan,0,0,15.0


In [6]:
# 4. Identify Missing Dates for April and May from the Trip Data
# and assign the corresponding records to the test set.
# ---------------------------
# Create a date column in both the trip data and the merged merged data.
trip_df['date'] = trip_df['start_time'].dt.date
merged_df['date'] = merged_df['time'].dt.date

# Generate the full date range for April and May 2015
april_may_range = pd.date_range(start="2015-04-01", end="2015-05-31", freq="D").date

# Get the unique trip dates in April and May (from the trip data)
dates_in_trip = set(trip_df[trip_df['start_time'].dt.month.isin([4, 5])]['date'])

# Identify missing dates (dates in April–May that have no trips)
missing_dates = {d for d in april_may_range if d not in dates_in_trip}

# ---------------------------
# 5. Label Records as Training or Test
# For each record in the merged (weather) data, assign 'test' if its date is one of the missing dates,
# otherwise 'train'. Note that even if a trip spans midnight, you always use the start date.
# ---------------------------
merged_df['set'] = merged_df['date'].apply(lambda d: 'test' if d in missing_dates else 'train')

In [7]:
merged_df['weekend'] = pd.to_datetime(merged_df['date']).dt.dayofweek.isin([5, 6]).astype(int) #Creating weekend column

In [8]:
# Split the Data into Train and Test Sets
train_df = merged_df[merged_df['set'] == 'train'] 
test_df = merged_df[merged_df['set'] == 'test']

In [9]:
test_df.head()

,time,index,temperature_C,humidity,windspeed_kph,wind_direction_degres,wind_direction,pressure_mBar,visibility_km,wind_chill_C,...,hour,time_of_day,month,month_str,bad_weather,severe_weather,trip_count,date,set,weekend
4360,2015-04-02 00:00:00,4360,3.0,81,22.2,270,West,1014,10.0,-1.7,...,0,LateNight,4,Apr,0,0,0.0,2015-04-02,test,0
4361,2015-04-02 00:30:00,4361,3.0,75,22.2,280,West,1013,10.0,-1.7,...,0,LateNight,4,Apr,0,0,0.0,2015-04-02,test,0
4362,2015-04-02 01:00:00,4362,4.0,70,20.4,290,WNW,1013,10.0,-0.2,...,1,LateNight,4,Apr,0,0,0.0,2015-04-02,test,0
4363,2015-04-02 01:30:00,4363,3.0,75,13.0,250,WSW,1014,10.0,-0.3,...,1,LateNight,4,Apr,0,0,0.0,2015-04-02,test,0
4364,2015-04-02 02:00:00,4364,3.0,75,20.4,230,SW,1014,10.0,-1.5,...,2,LateNight,4,Apr,0,0,0.0,2015-04-02,test,0


In [10]:
test_df.shape[0] # Number of Records in the Test Set

1437

In [11]:
test_df['trip_count'].isnull().sum()  

0

**Looks like it worked!!**

In [12]:
test_df.drop(columns=['set'], inplace=True) # Drop the 'set' and 'date' columns from the test set
train_df.drop(columns=['set'], inplace=True) # Drop the 'set' and 'date' columns from the train set

C:\Users\gsant\AppData\Local\Temp\ipykernel_7172\421185300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=['set'], inplace=True) # Drop the 'set' and 'date' columns from the test set
C:\Users\gsant\AppData\Local\Temp\ipykernel_7172\421185300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=['set'], inplace=True) # Drop the 'set' and 'date' columns from the train set


In [13]:
train_df.columns # Check the columns in the train_df

Index(['time', 'index', 'temperature_C', 'humidity', 'windspeed_kph',
       'wind_direction_degres', 'wind_direction', 'pressure_mBar',
       'visibility_km', 'wind_chill_C', 'fog', 'rain', 'snow', 'thunder',
       'temperature_bin', 'wind_chill_bin', 'humidity_bin', 'hour',
       'time_of_day', 'month', 'month_str', 'bad_weather', 'severe_weather',
       'trip_count', 'date', 'weekend'],
      dtype='object')

In [ ]:
# Save the train and test sets to CSV files
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)